<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50 = ResNet50(weights='imagenet')
features = model.predict(x)

```

Next you will need to remove the last layer from the ResNet model. Here, we loop over the layers to use the sequential API. There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities. 

```python
# Remote the Last Layer of ResNEt
ResNet50._layers.pop(0)

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in ResNet50.layers:
    model.add(layer)

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [ ]:
import numpy as np
import os

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential, Model # <- May Use
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten  # Conv2D, MaxPooling2D, Flatten etc are three new layers

So...I'm guessing we should be loading each image in individual then testing against the RESNET stuff...

### This is direct from JC himself

In [76]:
res = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
# Remote the Last Layer of ResNE

# make all resnet layers untrainable
for layer in res.layers:
    layer.trainable = False
    
# add your head on top
x = res.output
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)

/home/ygeun/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Here I'm going to try to do something related to loading all the images and processing them...but it's not working too well.

In [34]:
mtns_p = os.listdir('./data/mountain/')
mtns = []
frst_p = os.listdir('./data/forest/')
frst = []

In [35]:
def get_img(the_path):
    return image.load_img(the_path, target_size=(224,224))

In [72]:
def contained_img_detect(img):
    '''
    This basically loads the ResNet50 model standalone
    and detects a single image.
    '''
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    model = ResNet50(weights='imagenet')
    for layer in res.layers:
        layer.trainable = False
    features = model.predict(x)  # Raw softmax probablities
    results = decode_predictions(features, top=5)[0]
    return results

In [73]:
# NOT really working because of something to do with
#  `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 7, 7, 2048)
def free_img_detect(img_names, geo='mountain'):
    '''
    Load images from a folder and detect what each
    one is, can do multiple images
    '''
    # Load in the apropos files, I know, two paths, v ugly 
    pics = []
    results = []
    for f in img_names:
        pics.append(get_img(f'./data/{geo}/'+f))
    # Model stuff
    model = ResNet50(weights='imagenet', include_top=False)
    for l in res.layers:
        l.trainable = False
    # Now we loop through each image
    for p in pics:
        x = image.img_to_array(p)
        x = np.expand_dims(p, axis=0)
        x = preprocess_input(x)
        features = model.predict(x)
        results.append(decode_predictions(features, top=5)[0])
    
    return results

Standalone example

In [74]:
mountain = get_img('./data/mountain/'+mtns_p[1])
contained_img_detect(mountain)

[('n09193705', 'alp', 0.9762064),
 ('n09468604', 'valley', 0.012974796),
 ('n04228054', 'ski', 0.0029883524),
 ('n03792972', 'mountain_tent', 0.0022515615),
 ('n09246464', 'cliff', 0.0011685749)]

In [75]:
forest = get_img('./data/forest/'+frst_p[5])
contained_img_detect(forest)

[('n09332890', 'lakeside', 0.12751177),
 ('n02793495', 'barn', 0.1079641),
 ('n09193705', 'alp', 0.10634963),
 ('n09468604', 'valley', 0.10080068),
 ('n11879895', 'rapeseed', 0.077921145)]

... I mean that works right?

I tried to just loop it through and...it technically 'works' but like, it takes sooo long. Plus I think the point of this assignment is to understand the ResNet50 thing but...instructions are a little difficult to follow with the whole appending the ResNet50 model to your own.

### Test all images

In [49]:
def process_img(img):
    x = image.img_to_array(img)
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
m = []
f = []
geo1 = 'mountain'
geo2 = 'forest'

for f in mtns_p:
        m.append(process_img(get_img(f'./data/{geo1}/'+f)))
for x in frst_p:
        f.append(process_img(get_img(f'./data/{geo2}/'+x)))

Okay no idea what's happening going to try and follow instructions maybe...

In [63]:
X = m
# mountains are positive class
y = np.ones(len(X), dtype=np.int)

In [64]:
type(y)

numpy.ndarray

In [65]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [87]:
type(X)

list

In [99]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

res = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
# Next you will need to remove the last layer from the ResNet model. 
#Here, we loop over the layers to use the sequential API.
#There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities.

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in res.layers:
    model.add(Dense(12, activation='relu'))
    model.add(Flatten())

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))

/home/ygeun/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [100]:
# Compile Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
res.summary()

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language

### Using piexif to check metadata

In [103]:
!pip install piexif

In [156]:
import piexif 
import random

path = 'data/'
geo = ['mountain', 'forest']

for a in frst_p:
    url = path + geo[1] + '/' + a
    exif_b = piexif.dump(url)
    gps_ifd = {piexif.GPSIFD.GPSLatitude: random.randint(1,100),
               piexif.GPSIFD.GPSLongitude: random.randint(1,100)}
    exif_dict = {"GPS": gps_ifd}
    piexif.insert(exif_b, url)
    print(f"""{a}: {exif_b}\n{url}""")
    break

nat1266.jpg: b'Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00'
data/forest/nat1266.jpg


Data is probably in the Thumb.db thing...